# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.u

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355177927431                   -1.10    6.0    168ms
  2   -9.356780506048       -2.80       -1.44    1.0   83.0ms
  3   -9.357070882930       -3.54       -2.76    2.5    151ms
  4   -9.357119695298       -4.31       -2.98    7.8    523ms
  5   -9.357120021042       -6.49       -3.12    1.1   80.0ms
  6   -9.357120184208       -6.79       -3.28    2.2   93.4ms
  7   -9.357120277315       -7.03       -3.47    1.0   80.7ms
  8   -9.357120305710       -7.55       -3.63    1.0   80.2ms
  9   -9.357120319616       -7.86       -3.90    1.0   79.2ms
 10   -9.357120322511       -8.54       -4.09    1.9   93.5ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.74779148803                   -0.97    6.2    501ms
  2   -18.75477042116       -2.16       -1.34    2.0    258ms
  3   -18.79234753963       -1.43       -1.91    4.6    356ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│      2
│     14
│      2
│      5
│      2
│      2
│      1
│      4
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  4   -18.79259024348       -3.61       -2.15    4.0    362ms
  5   -18.79261297267       -4.64       -3.23    2.2    272ms
  6   -18.79262299832       -5.00       -3.45    8.4    571ms
  7   -18.79262408709       -5.96       -3.77    1.2    226ms
  8   -18.79262417243       -7.07       -4.13    1.9    256ms


In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.54999312201                   -0.84    9.8    1.91s
  2   -37.55728884326       -2.14       -1.22    1.4    710ms
  3   -37.56044057721       -2.50       -2.19    7.5    1.25s
  4   -37.56469352465       -2.37       -2.02   14.1    1.88s
  5   -37.56477036321       -4.11       -2.09    3.4    920ms
  6   -37.56492271847       -3.82       -2.46    1.4    802ms
  7   -37.56495659909       -4.47       -2.61    2.4    849ms
  8   -37.56497547238       -4.72       -2.85    1.5    750ms
  9   -37.56498432387       -5.05       -3.42    3.6    860ms
 10   -37.56498504055       -6.14       -3.69    4.0    1.37s
 11   -37.56498516592       -6.90       -4.00    4.9    952ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355140556033                   -1.09    6.0    159ms
  2   -9.356811149468       -2.78       -1.91    1.0   63.1ms
  3   -9.357095172599       -3.55       -2.63    6.8    152ms
  4   -9.357095628559       -6.34       -2.56    2.6   95.0ms
  5   -9.357120076227       -4.61       -3.62    1.0   61.6ms
  6   -9.357120267689       -6.72       -3.95    4.8    132ms
  7   -9.357120320358       -7.28       -4.47    1.6   71.4ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.55065784881                   -0.84    8.6    1.81s
  2   -37.49642600901   +   -1.27       -1.49    3.5    733ms
  3   +7.623137962493   +    1.65       -0.24   11.5    2.06s
  4   -37.50502140794        1.65       -1.50    8.5    2.12s
  5   -37.46198655151   +   -1.37       -1.49    3.2    978ms
  6   -37.09053641020   +   -0.43       -1.22    4.4    1.04s
  7   -37.54536682035       -0.34       -1.87    4.5    981ms
  8   -37.55177533629       -2.19       -1.91    2.9    1.16s
  9   -37.56299740910       -1.95       -2.16    2.0    772ms
 10   -37.56427900121       -2.89       -2.54    2.5    865ms
 11   -37.56497030382       -3.16       -2.97    3.5    954ms
 12   -37.56489591321   +   -4.13       -3.00    3.0    957ms
 13   -37.56497348727       -4.11       -3.35    3.6    906ms
 14   -37.56498296985       -5.02       -3.78    3.5    1.24s
 15   -37

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.